<a href="https://colab.research.google.com/github/salsaimon/proyecto_recursoshumanos/blob/main/preprocesamientoSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [96]:
#  pandas actualizado correctamente
!pip install --upgrade pandas

# importar pandas y sqlite3
import pandas as pd
import sqlite3 as sql

# Crear una conexión a la base de datos SQLite
conn = sql.connect('/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos/base.db')

# Consultar las primeras filas de cada tabla
print("\nDatos de la tabla 'retiros':")
df_retiros = pd.read_sql("SELECT * FROM retiros LIMIT 5", conn)
print(df_retiros)

print("\nDatos de la tabla 'general':")
df_general = pd.read_sql("SELECT * FROM general LIMIT 5", conn)
print(df_general)

print("\nDatos de la tabla 'employee':")
df_employee = pd.read_sql("SELECT * FROM employee LIMIT 5", conn)
print(df_employee)

print("\nDatos de la tabla 'manager':")
df_manager = pd.read_sql("SELECT * FROM manager LIMIT 5", conn)
print(df_manager)

# Realizar consultas SQL para análisis
print("\nConteo de 'Attrition' en la tabla 'retiros':")
attrition_count = pd.read_sql("SELECT Attrition, COUNT(*) FROM retiros GROUP BY Attrition", conn)
print(attrition_count)

print("\nConteo de 'BusinessTravel' en la tabla 'general':")
business_travel_count = pd.read_sql("SELECT BusinessTravel, COUNT(*) FROM general GROUP BY BusinessTravel", conn)
print(business_travel_count)

# Realizar más consultas según sea necesario
# Ejemplo: Conteo de 'JobSatisfaction' en la tabla 'employee'
print("\nConteo de 'JobSatisfaction' en la tabla 'employee':")
job_satisfaction_count = pd.read_sql("SELECT JobSatisfaction, COUNT(*) FROM employee GROUP BY JobSatisfaction", conn)
print(job_satisfaction_count)




Datos de la tabla 'retiros':
   index  EmployeeID Attrition       retirementDate retirementType  \
0      0           2       Yes  2016-10-15 00:00:00    Resignation   
1      1           7       Yes  2016-07-20 00:00:00    Resignation   
2      2          14       Yes  2016-11-08 00:00:00    Resignation   
3      3          29       Yes  2016-10-15 00:00:00    Resignation   
4      4          31       Yes  2016-02-08 00:00:00    Resignation   

  resignationReason  
0            Others  
1            Stress  
2            Others  
3            Others  
4            Salary  

Datos de la tabla 'general':
   index  Age     BusinessTravel              Department  DistanceFromHome  \
0      0   51      Travel_Rarely                   Sales                 6   
1      1   31  Travel_Frequently  Research & Development                10   
2      2   32  Travel_Frequently  Research & Development                17   
3      3   38         Non-Travel  Research & Development                 2 

In [97]:
# Obtener la lista de tablas en la base de datos
tables = [row[0] for row in conn.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()]

# Mostrar información sobre cada tabla y sus columnas
for table in tables:
    print(f"\nInformación sobre la tabla '{table}':")

    # Consultar las primeras filas de la tabla
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5", conn)
    print(f"\nPrimeras 5 filas de la tabla '{table}':")
    print(df)

    # Mostrar la estructura de la tabla (tipos de datos)
    print(f"\nEstructura de la tabla '{table}':")
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table})")
    structure = cur.fetchall()
    for column in structure:
        print(f"Columna: {column[1]}, Tipo: {column[2]}")



Información sobre la tabla 'retiros':

Primeras 5 filas de la tabla 'retiros':
   index  EmployeeID Attrition       retirementDate retirementType  \
0      0           2       Yes  2016-10-15 00:00:00    Resignation   
1      1           7       Yes  2016-07-20 00:00:00    Resignation   
2      2          14       Yes  2016-11-08 00:00:00    Resignation   
3      3          29       Yes  2016-10-15 00:00:00    Resignation   
4      4          31       Yes  2016-02-08 00:00:00    Resignation   

  resignationReason  
0            Others  
1            Stress  
2            Others  
3            Others  
4            Salary  

Estructura de la tabla 'retiros':
Columna: index, Tipo: INTEGER
Columna: EmployeeID, Tipo: INTEGER
Columna: Attrition, Tipo: TEXT
Columna: retirementDate, Tipo: TIMESTAMP
Columna: retirementType, Tipo: TEXT
Columna: resignationReason, Tipo: TEXT

Información sobre la tabla 'general':

Primeras 5 filas de la tabla 'general':
   index  Age     BusinessTravel        

In [98]:
## tratamiento de la tabla de retiros
# Contar registros por ID de empleado en la tabla de retiros
employee_id_count = pd.read_sql("SELECT EmployeeID, COUNT(*) as count FROM retiros GROUP BY EmployeeID", conn)

# Mostrar el resultado
print("\nConteo de registros por ID de empleado en la tabla 'retiros':")
print(employee_id_count)






Conteo de registros por ID de empleado en la tabla 'retiros':
     EmployeeID  count
0             2      1
1             7      1
2            14      1
3            29      1
4            31      1
..          ...    ...
645        3973      1
646        3974      1
647        3980      1
648        3981      1
649        3995      1

[650 rows x 2 columns]


In [99]:
# Resumen de la columna 'retirementDate'
print("\nResumen de la columna 'retirementDate' en la tabla 'retiros':")

# Contar valores no nulos y nulos
total_retiros = pd.read_sql("SELECT COUNT(*) FROM retiros WHERE retirementDate IS NOT NULL", conn).iloc[0, 0]
total_nulos = pd.read_sql("SELECT COUNT(*) FROM retiros WHERE retirementDate IS NULL", conn).iloc[0, 0]
print(f"Total de fechas de retiro no nulas: {total_retiros}")
print(f"Total de fechas de retiro nulas: {total_nulos}")

# Estadísticas descriptivas
retiros_df = pd.read_sql("SELECT retirementDate FROM retiros WHERE retirementDate IS NOT NULL", conn)
retiros_df['retirementDate'] = pd.to_datetime(retiros_df['retirementDate'])  # Convertir a datetime
print("\nEstadísticas descriptivas de 'retirementDate':")
print(f"Fecha mínima de retiro: {retiros_df['retirementDate'].min()}")
print(f"Fecha máxima de retiro: {retiros_df['retirementDate'].max()}")

# Distribución por año
retiros_df['Año'] = retiros_df['retirementDate'].dt.year
retiros_por_anio = retiros_df.groupby('Año').size().reset_index(name='Cantidad de retiros')
print("\nDistribución de retiros por año:")
print(retiros_por_anio)




Resumen de la columna 'retirementDate' en la tabla 'retiros':
Total de fechas de retiro no nulas: 650
Total de fechas de retiro nulas: 0

Estadísticas descriptivas de 'retirementDate':
Fecha mínima de retiro: 2016-01-02 00:00:00
Fecha máxima de retiro: 2016-12-16 00:00:00

Distribución de retiros por año:
    Año  Cantidad de retiros
0  2016                  650


In [100]:
# Eliminar los registros con fechas de retiro en el año 2015
conn.execute("DELETE FROM retiros WHERE strftime('%Y', retirementDate) = '2015'")
conn.commit()

# Verificar que los registros han sido eliminados
retiros_df = pd.read_sql("SELECT retirementDate FROM retiros WHERE retirementDate IS NOT NULL", conn)
retiros_df['retirementDate'] = pd.to_datetime(retiros_df['retirementDate'])  # Convertir a datetime
print("\nVerificación después de eliminar los datos del año 2015:")
print(f"Fecha mínima de retiro: {retiros_df['retirementDate'].min()}")
print(f"Fecha máxima de retiro: {retiros_df['retirementDate'].max()}")

# Distribución de retiros por año después de eliminar los datos
retiros_df['Año'] = retiros_df['retirementDate'].dt.year
retiros_por_anio = retiros_df.groupby('Año').size().reset_index(name='Cantidad de retiros')
print("\nDistribución de retiros por año después de eliminación:")
print(retiros_por_anio)


Verificación después de eliminar los datos del año 2015:
Fecha mínima de retiro: 2016-01-02 00:00:00
Fecha máxima de retiro: 2016-12-16 00:00:00

Distribución de retiros por año después de eliminación:
    Año  Cantidad de retiros
0  2016                  650


In [101]:
##analisis de retirementType

# Consultar los datos de la columna 'retirementType'
retiros_df = pd.read_sql("SELECT retirementType FROM retiros", conn)

# Contar valores únicos y frecuencia de cada tipo
print("\nResumen de la columna 'retirementType':")
print(f"Valores únicos en 'retirementType': {retiros_df['retirementType'].nunique()}")
print("\nFrecuencia de cada tipo de retiro:")
print(retiros_df['retirementType'].value_counts())

# Contar valores nulos
total_nulos = retiros_df['retirementType'].isnull().sum()
print(f"\nTotal de valores nulos en 'retirementType': {total_nulos}")


Resumen de la columna 'retirementType':
Valores únicos en 'retirementType': 2

Frecuencia de cada tipo de retiro:
retirementType
Resignation    585
Fired           65
Name: count, dtype: int64

Total de valores nulos en 'retirementType': 0


In [108]:
import pandas as pd

# Leer todos los datos de la tabla 'retiros'
retiros_df = pd.read_sql("SELECT * FROM retiros", conn)

# Crear una copia del DataFrame original para modificaciones
retiros_processed_df = retiros_df.copy()

# Codificación One-Hot para la columna 'retirementType' sin eliminar ninguna categoría
encoded_retiros_df = pd.get_dummies(retiros_processed_df['retirementType'], prefix='retirementType')

# Convertir los valores True/False a 0/1
encoded_retiros_df = encoded_retiros_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_retiros_df], axis=1)

# Eliminar la columna original 'retirementType' si ya no es necesaria
retiros_processed_df.drop('retirementType', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")


   index  EmployeeID Attrition       retirementDate resignationReason  \
0      0           2       Yes  2016-10-15 00:00:00            Others   
1      1           7       Yes  2016-07-20 00:00:00            Stress   
2      2          14       Yes  2016-11-08 00:00:00            Others   
3      3          29       Yes  2016-10-15 00:00:00            Others   
4      4          31       Yes  2016-02-08 00:00:00            Salary   

   retirementType_Fired  retirementType_Resignation  
0                     0                           1  
1                     0                           1  
2                     0                           1  
3                     0                           1  
4                     0                           1  

Número total de filas en el DataFrame procesado: 650


In [109]:

# Leer todos los datos de la tabla 'retiros'
retiros_df = pd.read_sql("SELECT resignationReason FROM retiros", conn)

# Resumen de la columna 'resignationReason'
unique_reasons = retiros_df['resignationReason'].unique()
reason_counts = retiros_df['resignationReason'].value_counts()
missing_values = retiros_df['resignationReason'].isnull().sum()

# Imprimir resumen
print(f"\nValores únicos en 'resignationReason': {unique_reasons}")
print("\nFrecuencia de cada motivo de renuncia:")
print(reason_counts)
print(f"\nNúmero total de valores nulos en 'resignationReason': {missing_values}")


Valores únicos en 'resignationReason': ['Others' 'Stress' 'Salary' None]

Frecuencia de cada motivo de renuncia:
resignationReason
Others    296
Salary    173
Stress    116
Name: count, dtype: int64

Número total de valores nulos en 'resignationReason': 65


In [110]:
# Reemplazar valores nulos con 'Unknown'
retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)

# Codificación One-Hot para la columna 'resignationReason'
encoded_reasons_df = pd.get_dummies(retiros_processed_df['resignationReason'], prefix='resignationReason', drop_first=False)

# Convertir valores True/False a 1/0 en las columnas codificadas
encoded_reasons_df = encoded_reasons_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  
0                         0                         0  
1                         0                         1  
2             

<ipython-input-110-b8b66264da45>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)


In [111]:
# Imprimir las primeras filas del DataFrame procesado
print("Primeras filas de retiros_processed_df:")
print(retiros_processed_df.head())

# Mostrar el resumen de las nuevas columnas de codificación One-Hot
print("\nResumen de las columnas codificadas para 'resignationReason':")
encoded_columns = [col for col in retiros_processed_df.columns if col.startswith('resignationReason')]
print(retiros_processed_df[encoded_columns].sum())

# Mostrar el número total de filas en el DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

Primeras filas de retiros_processed_df:
   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  
0                         0                         0  
1                         0   

In [112]:
# Verificar si la columna 'resignationReason' está en el DataFrame
if 'resignationReason' in retiros_processed_df.columns:
    # Reemplazar valores nulos con 'Unknown'
    retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)

    # Codificación One-Hot para la columna 'resignationReason'
    encoded_reasons_df = pd.get_dummies(retiros_processed_df['resignationReason'], prefix='resignationReason', drop_first=False)

    # Agregar las columnas codificadas al DataFrame procesado
    retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

    # Eliminar la columna original 'resignationReason'
    retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

    # Eliminar la columna 'Unknown' resultante de la codificación
    if 'resignationReason_Unknown' in retiros_processed_df.columns:
        retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)
else:
    print("La columna 'resignationReason' no está en el DataFrame.")

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")


La columna 'resignationReason' no está en el DataFrame.
   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  
0                         0                         0  
1             

In [113]:
# Convertir valores True/False a 1/0 en las columnas codificadas
encoded_reasons_df = encoded_reasons_df.astype(int)

# Verificar si las columnas codificadas ya están en el DataFrame
for col in encoded_reasons_df.columns:
    if col in retiros_processed_df.columns:
        # Eliminar las columnas duplicadas
        retiros_processed_df.drop(col, axis=1, inplace=True)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna 'Unknown' resultante de la codificación si existe
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  
0                         0                         0  
1                         0                         1  
2             

In [114]:
# Convertir valores True/False a 1/0 en las columnas especificadas
columns_to_convert = ['resignationReason_Others', 'resignationReason_Salary', 'resignationReason_Stress']
retiros_processed_df[columns_to_convert] = retiros_processed_df[columns_to_convert].astype(int)

# Imprimir el DataFrame procesado para verificar
print(retiros_processed_df.head())

   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  
0                         0                         0  
1                         0                         1  
2             

In [115]:
# Resumen de la columna 'Attrition'
attrition_summary = retiros_processed_df['Attrition'].value_counts()
total_count = len(retiros_processed_df)

# Imprimir el resumen
print("Resumen de la variable 'Attrition':")
print(attrition_summary)
print(f"\nNúmero total de registros en 'Attrition': {total_count}")
print(f"Proporción de cada categoría en 'Attrition':")
print(attrition_summary / total_count)

Resumen de la variable 'Attrition':
Attrition
Yes    650
Name: count, dtype: int64

Número total de registros en 'Attrition': 650
Proporción de cada categoría en 'Attrition':
Attrition
Yes    1.0
Name: count, dtype: float64


In [116]:
# Eliminar la columna 'Attrition' del DataFrame procesado
retiros_processed_df.drop('Attrition', axis=1, inplace=True)

# Imprimir el DataFrame procesado para verificar los cambios
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

   index  EmployeeID       retirementDate  retirementType_Fired  \
0      0           2  2016-10-15 00:00:00                     0   
1      1           7  2016-07-20 00:00:00                     0   
2      2          14  2016-11-08 00:00:00                     0   
3      3          29  2016-10-15 00:00:00                     0   
4      4          31  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  
0                         0                         0  
1                         0                         1  
2                         0                         0  
3                 

In [117]:
# Leer los datos de la tabla 'general'
general_df = pd.read_sql("SELECT * FROM general", conn)

# Imprimir las primeras filas del DataFrame
print("Primeras filas de la tabla 'general':")
print(general_df.head())

# Obtener información general sobre el DataFrame
print("\nInformación general de la tabla 'general':")
print(general_df.info())

# Resumen estadístico de las columnas numéricas
print("\nResumen estadístico de las columnas numéricas:")
print(general_df.describe())

# Resumen de las columnas categóricas
print("\nResumen de las columnas categóricas:")
for col in general_df.select_dtypes(include=['object']).columns:
    print(f"\nResumen de la columna '{col}':")
    print(general_df[col].value_counts())

Primeras filas de la tabla 'general':
   index  Age     BusinessTravel              Department  DistanceFromHome  \
0      0   51      Travel_Rarely                   Sales                 6   
1      1   31  Travel_Frequently  Research & Development                10   
2      2   32  Travel_Frequently  Research & Development                17   
3      3   38         Non-Travel  Research & Development                 2   
4      4   32      Travel_Rarely  Research & Development                10   

   Education EducationField  EmployeeCount  EmployeeID  Gender  ...  \
0          2  Life Sciences              1           1  Female  ...   
1          1  Life Sciences              1           2  Female  ...   
2          4          Other              1           3    Male  ...   
3          5  Life Sciences              1           4    Male  ...   
4          1        Medical              1           5    Male  ...   

   NumCompaniesWorked PercentSalaryHike StandardHours  StockOption

In [118]:
# Contar el número de registros por EmployeeID
employee_record_counts = general_df.groupby('EmployeeID').size()

# Mostrar los primeros registros de la cuenta por empleado
print(employee_record_counts.head())

EmployeeID
1    2
2    1
3    2
4    2
5    2
dtype: int64


In [119]:
# Crear una copia del DataFrame original
generalprocessed_df = general_df.copy()

# Agrupar por EmployeeID y calcular el promedio de las columnas numéricas
numeric_cols = generalprocessed_df.select_dtypes(include='number').columns

# Excluir 'EmployeeID' de las columnas numéricas para evitar conflictos
numeric_cols = numeric_cols.difference(['EmployeeID'])
avg_data = generalprocessed_df.groupby('EmployeeID')[numeric_cols].mean().reset_index()

# Mantener la información no numérica utilizando la moda
categorical_cols = generalprocessed_df.select_dtypes(include='object').columns

# Excluir 'EmployeeID' de las columnas categóricas
categorical_cols = categorical_cols.difference(['EmployeeID'])
mode_data = generalprocessed_df.groupby('EmployeeID')[categorical_cols].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').reset_index()

# Unir los datos promediados con los datos categóricos
generalprocessed_df = avg_data.merge(mode_data, on='EmployeeID')

# Imprimir las primeras filas del nuevo DataFrame
print(generalprocessed_df.head())

# Mostrar información general del nuevo DataFrame
print(generalprocessed_df.info())

   EmployeeID   Age  DistanceFromHome  Education  EmployeeCount  JobLevel  \
0           1  51.0               6.0        2.0            1.0       1.0   
1           2  31.0              10.0        1.0            1.0       1.0   
2           3  32.0              17.0        4.0            1.0       4.0   
3           4  38.0               2.0        5.0            1.0       3.0   
4           5  32.0              10.0        1.0            1.0       1.0   

   MonthlyIncome  NumCompaniesWorked  PercentSalaryHike  StandardHours  ...  \
0       131160.0                 1.0               11.0            8.0  ...   
1        41890.0                 0.0               23.0            8.0  ...   
2       193280.0                 1.0               15.0            8.0  ...   
3        83210.0                 3.0               11.0            8.0  ...   
4        23420.0                 4.0               12.0            8.0  ...   

   YearsSinceLastPromotion  YearsWithCurrManager   index     B

In [120]:

# Verificar la cantidad de registros únicos por EmployeeID
print("Número único de EmployeeID:", generalprocessed_df['EmployeeID'].nunique())
print("Número total de registros en generalprocessed_df:", generalprocessed_df.shape[0])

# Verificar datos nulos
print("\nDatos nulos en generalprocessed_df:")
print(generalprocessed_df.isnull().sum())

# Revisar estadísticas descriptivas
print("\nResumen estadístico de generalprocessed_df:")
print(generalprocessed_df.describe(include='all'))

# Validar columnas de texto
print("\nConteo de valores únicos en columnas categóricas:")
print(generalprocessed_df[['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus']].value_counts())

# Validar datos de tiempo
print("\nConteo de valores en InfoDate:")
print(generalprocessed_df['InfoDate'].value_counts())

# Comparar con datos originales (seleccionando algunos registros aleatorios)
print("\nComparación de registros aleatorios:")
sample_original = general_df.sample(5)
sample_processed = generalprocessed_df[generalprocessed_df['EmployeeID'].isin(sample_original['EmployeeID'])]

print("\nMuestra de datos originales:")
print(sample_original)
print("\nMuestra de datos procesados:")
print(sample_processed)

Número único de EmployeeID: 5060
Número total de registros en generalprocessed_df: 5060

Datos nulos en generalprocessed_df:
EmployeeID                  0
Age                         0
DistanceFromHome            0
Education                   0
EmployeeCount               0
JobLevel                    0
MonthlyIncome               0
NumCompaniesWorked         23
PercentSalaryHike           0
StandardHours               0
StockOptionLevel            0
TotalWorkingYears          11
TrainingTimesLastYear       0
YearsAtCompany              0
YearsSinceLastPromotion     0
YearsWithCurrManager        0
index                       0
BusinessTravel              0
Department                  0
EducationField              0
Gender                      0
InfoDate                    0
JobRole                     0
MaritalStatus               0
dtype: int64

Resumen estadístico de generalprocessed_df:
         EmployeeID          Age  DistanceFromHome    Education  \
count   5060.000000  5060.0000

In [121]:
from sklearn.impute import SimpleImputer

# Imputar valores nulos con la media
generalprocessed_df['NumCompaniesWorked'].fillna(generalprocessed_df['NumCompaniesWorked'].mean(), inplace=True)
generalprocessed_df['TotalWorkingYears'].fillna(generalprocessed_df['TotalWorkingYears'].mean(), inplace=True)

# Verificar si aún hay valores nulos
print("Datos nulos después de imputar:")
print(generalprocessed_df.isnull().sum())

Datos nulos después de imputar:
EmployeeID                 0
Age                        0
DistanceFromHome           0
Education                  0
EmployeeCount              0
JobLevel                   0
MonthlyIncome              0
NumCompaniesWorked         0
PercentSalaryHike          0
StandardHours              0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
YearsAtCompany             0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
index                      0
BusinessTravel             0
Department                 0
EducationField             0
Gender                     0
InfoDate                   0
JobRole                    0
MaritalStatus              0
dtype: int64


<ipython-input-121-a01f3461165b>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  generalprocessed_df['NumCompaniesWorked'].fillna(generalprocessed_df['NumCompaniesWorked'].mean(), inplace=True)
<ipython-input-121-a01f3461165b>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [122]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# 1. Eliminar columnas no necesarias
columns_to_drop = ['EmployeeCount', 'StockOptionLevel', 'StandardHours']
generalprocessed_df = generalprocessed_df.drop(columns=columns_to_drop)

# 2. Estandarización de variables numéricas
numerical_columns = [
    'Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
    'PercentSalaryHike', 'TotalWorkingYears',
    'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion',
    'YearsWithCurrManager'
]

# Inicializar el estandarizador
scaler = StandardScaler()

# Aplicar estandarización
generalprocessed_df[numerical_columns] = scaler.fit_transform(generalprocessed_df[numerical_columns])

# 3. Codificación de variables categóricas ordinales
ordinal_columns = {
    'Education': [1, 2, 3, 4, 5],
    'JobLevel': [1, 2, 3, 4, 5]
}

# Reemplazar categorías ordinales con valores numéricos
for col, order in ordinal_columns.items():
    generalprocessed_df[col] = pd.Categorical(generalprocessed_df[col], categories=order, ordered=True).codes

# 4. Codificación de variables categóricas no ordinales (One-Hot Encoding)
categorical_columns = [
    'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus'
]

# Crear el codificador OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False, drop='first')

# Aplicar el codificador a las columnas categóricas
one_hot_encoded = one_hot_encoder.fit_transform(generalprocessed_df[categorical_columns])

# Crear un DataFrame con las columnas codificadas
one_hot_df = pd.DataFrame(one_hot_encoded, columns=one_hot_encoder.get_feature_names_out(categorical_columns))

# Unir el DataFrame original con el DataFrame codificado
generalprocessed = pd.concat([generalprocessed_df.drop(categorical_columns, axis=1), one_hot_df], axis=1)

# Verificar el DataFrame final
print(generalprocessed.head())

   EmployeeID       Age  DistanceFromHome  Education  JobLevel  MonthlyIncome  \
0           1  1.565878         -0.394163          1         0       1.424580   
1           2 -0.591572          0.101202          0         0      -0.484657   
2           3 -0.483699          0.968092          3         3       2.753154   
3           4  0.163536         -0.889529          4         2       0.399063   
4           5 -0.483699          0.101202          0         0      -0.879679   

   NumCompaniesWorked  PercentSalaryHike  TotalWorkingYears  \
0           -0.684418          -1.156044          -1.273857   
1           -1.081723           2.116449          -0.629850   
2           -0.684418          -0.065213          -0.758651   
3            0.110192          -1.156044           0.271761   
4            0.507496          -0.883336          -0.243445   

   TrainingTimesLastYear  ...  JobRole_Human Resources  \
0               2.529894  ...                      0.0   
1               0.

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [123]:
# Mostrar las primeras 10 filas del DataFrame 'generalprocessed'
print("Primeros 10 registros del DataFrame 'generalprocessed':")
print(generalprocessed.head(10))

# Mostrar información sobre el DataFrame
print("\nInformación del DataFrame 'generalprocessed':")
print(generalprocessed.info())

# Mostrar un resumen estadístico del DataFrame
print("\nResumen estadístico del DataFrame 'generalprocessed':")
print(generalprocessed.describe(include='all'))

Primeros 10 registros del DataFrame 'generalprocessed':
   EmployeeID       Age  DistanceFromHome  Education  JobLevel  MonthlyIncome  \
0           1  1.565878         -0.394163          1         0       1.424580   
1           2 -0.591572          0.101202          0         0      -0.484657   
2           3 -0.483699          0.968092          3         3       2.753154   
3           4  0.163536         -0.889529          4         2       0.399063   
4           5 -0.483699          0.101202          0         0      -0.879679   
5           6  1.026515         -0.146480          2         3      -0.509894   
6           7 -0.915189          0.225044          1         1      -0.137328   
7           8 -0.807317          1.091933          2         1      -0.708367   
8           9 -0.591572         -1.013370          2         2      -0.943413   
9          10 -1.238807         -0.270322          3         3       1.499008   

   NumCompaniesWorked  PercentSalaryHike  TotalWorki

In [124]:
# Leer todos los datos de la tabla 'employee'
query = "SELECT * FROM employee"
df_employee = pd.read_sql_query(query, conn)

# Mostrar información sobre el DataFrame actualizado
print("\nInformación del DataFrame 'df_employee':")
print(df_employee.info())

# Mostrar un resumen estadístico del DataFrame actualizado
print("\nResumen estadístico del DataFrame 'df_employee':")
print(df_employee.describe(include='all'))


Información del DataFrame 'df_employee':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    8820 non-null   int64  
 1   EmployeeID               8820 non-null   int64  
 2   EnvironmentSatisfaction  8770 non-null   float64
 3   JobSatisfaction          8780 non-null   float64
 4   WorkLifeBalance          8744 non-null   float64
 5   DateSurvey               8820 non-null   object 
dtypes: float64(3), int64(2), object(1)
memory usage: 413.6+ KB
None

Resumen estadístico del DataFrame 'df_employee':
              index   EmployeeID  EnvironmentSatisfaction  JobSatisfaction  \
count   8820.000000  8820.000000              8770.000000      8780.000000   
unique          NaN          NaN                      NaN              NaN   
top             NaN          NaN                      NaN          

In [125]:
# Crear una copia del DataFrame original
df_employee_processed = df_employee.copy()

# Agrupar por EmployeeID y calcular el promedio de las columnas numéricas
numeric_cols = df_employee_processed.select_dtypes(include='number').columns

# Excluir 'EmployeeID' de las columnas numéricas para evitar conflictos
numeric_cols = numeric_cols.difference(['EmployeeID'])
avg_data = df_employee_processed.groupby('EmployeeID')[numeric_cols].mean().reset_index()

# Mantener la información no numérica utilizando la moda
categorical_cols = df_employee_processed.select_dtypes(include='object').columns

# Excluir 'EmployeeID' de las columnas categóricas
categorical_cols = categorical_cols.difference(['EmployeeID'])
mode_data = df_employee_processed.groupby('EmployeeID')[categorical_cols].agg(
    lambda x: x.mode()[0] if not x.mode().empty else 'Unknown'
).reset_index()

# Unir los datos promediados con los datos categóricos
df_employee_processed = avg_data.merge(mode_data, on='EmployeeID')

# Imprimir las primeras filas del nuevo DataFrame
print(df_employee_processed.head())

# Mostrar información general del nuevo DataFrame
print(df_employee_processed.info())

   EmployeeID  EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance  \
0           1                      3.0              4.0              2.0   
1           2                      3.0              2.0              4.0   
2           3                      2.0              2.0              1.0   
3           4                      4.0              4.0              3.0   
4           5                      4.0              1.0              3.0   

    index           DateSurvey  
0  2205.0  2015-12-31 00:00:00  
1     1.0  2015-12-31 00:00:00  
2  2207.0  2015-12-31 00:00:00  
3  2208.0  2015-12-31 00:00:00  
4  2209.0  2015-12-31 00:00:00  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5060 entries, 0 to 5059
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmployeeID               5060 non-null   int64  
 1   EnvironmentSatisfaction  5030 non-null   float64
 2   JobSatisfaction   

In [126]:
# Calcular la media de cada columna numérica
mean_values = df_employee_processed[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']].mean()

# Reemplazar los valores nulos con la media correspondiente
df_employee_processed[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']] = df_employee_processed[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']].fillna(mean_values)

# Verificar que ya no hay valores nulos en estas columnas
print("\nDatos nulos después de la imputación:")
print(df_employee_processed[['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']].isnull().sum())


Datos nulos después de la imputación:
EnvironmentSatisfaction    0
JobSatisfaction            0
WorkLifeBalance            0
dtype: int64


In [127]:
from sklearn.preprocessing import OrdinalEncoder
import pandas as pd

# Mantener las columnas que no se deben modificar
df_model = df_employee_processed[['EmployeeID', 'DateSurvey']].copy()

# Definir las columnas categóricas ordinales
ordinal_cols = ['EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance']

# Convertir las columnas ordinales a números (si no lo están ya)
df_employee_processed[ordinal_cols] = df_employee_processed[ordinal_cols].apply(pd.to_numeric, errors='coerce')

# Redondear los valores en las columnas ordinales para asegurarse de que estén en el rango de 1 a 5
df_employee_processed[ordinal_cols] = df_employee_processed[ordinal_cols].round().astype(int)

# Verificar los valores únicos en cada columna ordinal
for col in ordinal_cols:
    unique_values = df_employee_processed[col].dropna().unique()
    print(f"Unique values in {col}: {unique_values}")

# Instanciar el codificador OrdinalEncoder, con categorías de 1 a 5
categories = [list(range(1, 6))] * len(ordinal_cols)
ordinal_encoder = OrdinalEncoder(categories=categories)

# Aplicar OrdinalEncoder a las columnas categóricas ordinales
encoded_ordinal_data = ordinal_encoder.fit_transform(df_employee_processed[ordinal_cols])

# Crear un DataFrame con las columnas codificadas ordinalmente
encoded_ordinal_df = pd.DataFrame(encoded_ordinal_data, columns=ordinal_cols)

# Asegurarse de que los valores sigan siendo enteros de 1 a 5
encoded_ordinal_df = encoded_ordinal_df.astype(int)

# Eliminar las columnas originales que serán reemplazadas
df_employee_processed = df_employee_processed.drop(columns=ordinal_cols)

# Añadir las columnas codificadas al DataFrame original
df_employee_processed = pd.concat([df_model, encoded_ordinal_df], axis=1)

# Mostrar el DataFrame final
print(df_employee_processed.head())

Unique values in EnvironmentSatisfaction: [3 2 4 1]
Unique values in JobSatisfaction: [4 2 1 3]
Unique values in WorkLifeBalance: [2 4 1 3]
   EmployeeID           DateSurvey  EnvironmentSatisfaction  JobSatisfaction  \
0           1  2015-12-31 00:00:00                        2                3   
1           2  2015-12-31 00:00:00                        2                1   
2           3  2015-12-31 00:00:00                        1                1   
3           4  2015-12-31 00:00:00                        3                3   
4           5  2015-12-31 00:00:00                        3                0   

   WorkLifeBalance  
0                1  
1                3  
2                0  
3                2  
4                2  


In [128]:
# Leer todos los datos de la tabla 'manager'
query = "SELECT * FROM manager"
df_manager = pd.read_sql_query(query, conn)

# Mostrar información sobre el DataFrame actualizado
print("\nInformación del DataFrame 'df_manager':")
print(df_manager.info())

# Mostrar un resumen estadístico del DataFrame actualizado
print("\nResumen estadístico del DataFrame 'df_manager':")
print(df_manager.describe(include='all'))


Información del DataFrame 'df_manager':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8820 entries, 0 to 8819
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              8820 non-null   int64 
 1   EmployeeID         8820 non-null   int64 
 2   JobInvolvement     8820 non-null   int64 
 3   PerformanceRating  8820 non-null   int64 
 4   SurveyDate         8820 non-null   object
dtypes: int64(4), object(1)
memory usage: 344.7+ KB
None

Resumen estadístico del DataFrame 'df_manager':
              index   EmployeeID  JobInvolvement  PerformanceRating  \
count   8820.000000  8820.000000     8820.000000        8820.000000   
unique          NaN          NaN             NaN                NaN   
top             NaN          NaN             NaN                NaN   
freq            NaN          NaN             NaN                NaN   
mean    4409.500000  2591.500000        2.729932           3.15

In [129]:
df_manager_processed = df_manager.copy()

# Agrupar por EmployeeID y calcular el promedio de las columnas numéricas
numeric_cols = df_manager_processed.select_dtypes(include='number').columns

# Excluir 'EmployeeID' de las columnas numéricas para evitar conflictos
numeric_cols = numeric_cols.difference(['EmployeeID'])
avg_data = df_manager_processed.groupby('EmployeeID')[numeric_cols].mean().reset_index()

# Mantener la información no numérica utilizando la moda
categorical_cols = df_manager_processed.select_dtypes(include='object').columns

# Excluir 'EmployeeID' de las columnas categóricas
categorical_cols = categorical_cols.difference(['EmployeeID'])
mode_data = df_manager_processed.groupby('EmployeeID')[categorical_cols].agg(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown').reset_index()

# Unir los datos promediados con los datos categóricos
df_manager_processed = avg_data.merge(mode_data, on='EmployeeID')

# Imprimir las primeras filas del nuevo DataFrame
print(df_manager_processed.head())

# Mostrar información general del nuevo DataFrame
print(df_manager_processed.info())

   EmployeeID  JobInvolvement  PerformanceRating   index           SurveyDate
0           1             3.0                3.0  2205.0  2015-12-31 00:00:00
1           2             2.0                4.0     1.0  2015-12-31 00:00:00
2           3             3.0                3.0  2207.0  2015-12-31 00:00:00
3           4             2.0                3.0  2208.0  2015-12-31 00:00:00
4           5             3.0                3.0  2209.0  2015-12-31 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5060 entries, 0 to 5059
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   EmployeeID         5060 non-null   int64  
 1   JobInvolvement     5060 non-null   float64
 2   PerformanceRating  5060 non-null   float64
 3   index              5060 non-null   float64
 4   SurveyDate         5060 non-null   object 
dtypes: float64(3), int64(1), object(1)
memory usage: 197.8+ KB
None


In [130]:
# Mantener las columnas que no se deben modificar
df_model_manager = df_manager_processed[['EmployeeID', 'SurveyDate']].copy()

# Revisar los valores únicos en las columnas ordinales
print("Valores únicos en 'JobInvolvement':", df_manager_processed['JobInvolvement'].unique())
print("Valores únicos en 'PerformanceRating':", df_manager_processed['PerformanceRating'].unique())

# Reemplazar valores nulos por la media o mediana si es necesario
df_manager_processed['JobInvolvement'].fillna(df_manager_processed['JobInvolvement'].median(), inplace=True)
df_manager_processed['PerformanceRating'].fillna(df_manager_processed['PerformanceRating'].median(), inplace=True)

# Asegurarse de que los valores estén en el rango de 1 a 5
df_manager_processed['JobInvolvement'] = df_manager_processed['JobInvolvement'].clip(lower=1, upper=5)
df_manager_processed['PerformanceRating'] = df_manager_processed['PerformanceRating'].clip(lower=1, upper=5)

# Definir las columnas categóricas ordinales
ordinal_cols_manager = ['JobInvolvement', 'PerformanceRating']

# Instanciar el codificador OrdinalEncoder, con categorías de 1 a 5
ordinal_encoder_manager = OrdinalEncoder(categories=[range(1, 6), range(1, 6)])

# Aplicar OrdinalEncoder a las columnas categóricas ordinales
encoded_ordinal_data_manager = ordinal_encoder_manager.fit_transform(df_manager_processed[ordinal_cols_manager])

# Crear un DataFrame con las columnas codificadas ordinalmente
encoded_ordinal_df_manager = pd.DataFrame(encoded_ordinal_data_manager, columns=ordinal_cols_manager)

# Asegurarse de que los valores sigan siendo enteros de 1 a 5
encoded_ordinal_df_manager = encoded_ordinal_df_manager.astype(int)

# Mantener las columnas que no se deben modificar
df_model_manager = df_manager_processed[['EmployeeID', 'SurveyDate']].copy()

# Añadir las columnas codificadas al DataFrame original
df_model_manager = pd.concat([df_model_manager, encoded_ordinal_df_manager], axis=1)

# Guardar el DataFrame final en df_manager_processed
df_manager_processed = df_model_manager

# Mostrar el DataFrame final
print(df_manager_processed.head())

Valores únicos en 'JobInvolvement': [3. 2. 1. 4.]
Valores únicos en 'PerformanceRating': [3. 4.]
   EmployeeID           SurveyDate  JobInvolvement  PerformanceRating
0           1  2015-12-31 00:00:00               2                  2
1           2  2015-12-31 00:00:00               1                  3
2           3  2015-12-31 00:00:00               2                  2
3           4  2015-12-31 00:00:00               1                  2
4           5  2015-12-31 00:00:00               2                  2


<ipython-input-130-bab8b1b90fe0>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_manager_processed['JobInvolvement'].fillna(df_manager_processed['JobInvolvement'].median(), inplace=True)
<ipython-input-130-bab8b1b90fe0>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [131]:
# Información general del DataFrame 'df_manager_processed'
print("\nInformación del DataFrame 'df_manager_processed':")
print(df_manager_processed.info())

# Resumen estadístico del DataFrame 'df_manager_processed'
print("\nResumen estadístico del DataFrame 'df_manager_processed':")
print(df_manager_processed.describe(include='all'))

# Imprimir las primeras 10 filas del DataFrame 'df_manager_processed'
print("\nPrimeras 10 filas del DataFrame 'df_manager_processed':")
print(df_manager_processed.head(10))

# Conteo de valores únicos en cada columna
print("\nConteo de valores únicos en cada columna:")
for col in df_manager_processed.columns:
    print(f"{col}:")
    print(df_manager_processed[col].value_counts())
    print()


Información del DataFrame 'df_manager_processed':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5060 entries, 0 to 5059
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   EmployeeID         5060 non-null   int64 
 1   SurveyDate         5060 non-null   object
 2   JobInvolvement     5060 non-null   int64 
 3   PerformanceRating  5060 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 158.2+ KB
None

Resumen estadístico del DataFrame 'df_manager_processed':
         EmployeeID           SurveyDate  JobInvolvement  PerformanceRating
count   5060.000000                 5060     5060.000000        5060.000000
unique          NaN                    2             NaN                NaN
top             NaN  2015-12-31 00:00:00             NaN                NaN
freq            NaN                 4410             NaN                NaN
mean    2799.119763                  NaN        1.726087         

In [132]:
import sqlite3

# Conectar a la nueva base de datos
conn = sqlite3.connect('baseparamodelo.db')

# Guardar DataFrames en la base de datos
df_manager_processed.to_sql('manager_processed', conn, if_exists='replace', index=False)
df_employee_processed.to_sql('employee_processed', conn, if_exists='replace', index=False)
generalprocessed.to_sql('general_processed', conn, if_exists='replace', index=False)
retiros_processed_df.to_sql('retiros_processed', conn, if_exists='replace', index=False)

# Cerrar la conexión
conn.close()

In [133]:
import sqlite3
import pandas as pd

# Conectar a la base de datos
conn = sqlite3.connect('baseparamodelo.db')

# Obtener el listado de tablas en la base de datos
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)

# Crear un diccionario para almacenar el resumen
summary = {}

# Para cada tabla, obtener el número de registros y mostrar un resumen de las columnas
for table in tables['name']:
    # Contar el número de registros en cada tabla
    count_query = f"SELECT COUNT(*) as count FROM {table};"
    count = pd.read_sql(count_query, conn).iloc[0]['count']

    # Obtener el resumen de las columnas
    columns_query = f"PRAGMA table_info({table});"
    columns_info = pd.read_sql(columns_query, conn)

    # Almacenar el resumen
    summary[table] = {
        'num_records': count,
        'columns': columns_info[['name', 'type']].to_dict('records')
    }

# Cerrar la conexión
conn.close()

# Mostrar el resumen
for table, info in summary.items():
    print(f"Tabla: {table}")
    print(f"Número de registros: {info['num_records']}")
    print("Columnas:")
    for column in info['columns']:
        print(f" - {column['name']} ({column['type']})")
    print("\n")


Tabla: manager_processed
Número de registros: 5060
Columnas:
 - EmployeeID (INTEGER)
 - SurveyDate (TEXT)
 - JobInvolvement (INTEGER)
 - PerformanceRating (INTEGER)


Tabla: employee_processed
Número de registros: 5060
Columnas:
 - EmployeeID (INTEGER)
 - DateSurvey (TEXT)
 - EnvironmentSatisfaction (INTEGER)
 - JobSatisfaction (INTEGER)
 - WorkLifeBalance (INTEGER)


Tabla: general_processed
Número de registros: 5060
Columnas:
 - EmployeeID (INTEGER)
 - Age (REAL)
 - DistanceFromHome (REAL)
 - Education (INTEGER)
 - JobLevel (INTEGER)
 - MonthlyIncome (REAL)
 - NumCompaniesWorked (REAL)
 - PercentSalaryHike (REAL)
 - TotalWorkingYears (REAL)
 - TrainingTimesLastYear (REAL)
 - YearsAtCompany (REAL)
 - YearsSinceLastPromotion (REAL)
 - YearsWithCurrManager (REAL)
 - index (REAL)
 - InfoDate (TEXT)
 - BusinessTravel_Travel_Frequently (REAL)
 - BusinessTravel_Travel_Rarely (REAL)
 - Department_Research & Development (REAL)
 - Department_Sales (REAL)
 - EducationField_Life Sciences (REAL)
